# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
file_path_list = []
for root, dirs, files in os.walk(filepath):
    file_path_list += glob.glob(os.path.join(root,'*'))

/Users/antoniolechuga/Developer/udacity/data-engineering/dend-p2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # removing the header line
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_full.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow([
        'artist',
        'firstName',
        'gender',
        'itemInSession',
        'lastName',
        'length',
        'level',
        'location',
        'sessionId',
        'song',
        'userId'
    ])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_full.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=tomato>event_datafile_full.csv</font>, located within the Workspace directory.  The event_datafile_full.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(auth_provider=auth_provider)

# To establish connection and begin executing queries, need a session
try:
    session = cluster.connect()
except Exception as e:
    print(f"Exception encountered while trying to connect with the db backend: {e}")

#### Create Keyspace

In [6]:
keyspace_config = "{'class': 'SimpleStrategy', 'replication_factor': 1}"
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS sparkifydb WITH REPLICATION = " + keyspace_config)
except Exception as e:
    print(f"Exception encountered while trying to create keyspace: {e}")

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(f"Exception encountered while trying to set keyspace: {e}")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
# creating table for query 1
query_1_table_create = """
    CREATE TABLE IF NOT EXISTS music_library_1 (
        artist text,
        title text,
        length float,
        session_id int,
        item_in_session int,
        PRIMARY KEY (session_id, item_in_session)
    );
"""

try:
    session.execute(query_1_table_create)
    print("Table 1 created successfully...")
except Exception as e:
    print(f"Exception encountered while creating table 1: {e}")

Table 1 created successfully...


In [9]:
# opening csv
file = 'event_datafile_full.csv'
query_1_table_insert = """
    INSERT INTO music_library_1 (artist, title, length, session_id, item_in_session)
    VALUES (%s, %s, %s, %s, %s);
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    for line in csvreader:
        # database insertion
        session.execute(query_1_table_insert, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
select_query_1 = """
    SELECT artist, title, length FROM music_library_1 
    WHERE session_id = 338 AND item_in_session = 4;
"""
try:
    rows = session.execute(select_query_1)
    for row in rows:
        print(f"{row.artist}, {row.title}, {row.length}")
except Exception as e:
    print(f"Exception encountered while querying table 1: {e}")

Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# creating table for query 2
query_2_table_create = """
    CREATE TABLE IF NOT EXISTS music_library_2 (
        artist text,
        title text,
        first_name text,
        last_name text,
        user_id int,
        session_id int,
        item_in_session int,
        PRIMARY KEY (user_id, session_id, item_in_session)
    );
"""

try:
    session.execute(query_2_table_create)
    print("Table 2 created successfully...")
except Exception as e:
    print(f"Exception encountered while creating table 2: {e}")

Table 2 created successfully...


In [12]:
# opening csv
file = 'event_datafile_full.csv'
query_2_table_insert = """
    INSERT INTO music_library_2 (artist, title, first_name, last_name, user_id, session_id, item_in_session)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    for line in csvreader:
        # database insertion
        session.execute(query_2_table_insert, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
select_query_2 = """
    SELECT artist, title, first_name, last_name FROM music_library_2 
    WHERE user_id = 10 AND session_id = 182;
"""
try:
    rows = session.execute(select_query_2)
    for row in rows:
        print(f"{row.artist}, {row.title}, {row.first_name}, {row.last_name}")
except Exception as e:
    print(f"Exception encountered while querying table 3: {e}")

Down To The Bone, Keep On Keepin' On, Sylvie, Cruz
Three Drives, Greece 2000, Sylvie, Cruz
Sebastien Tellier, Kilometer, Sylvie, Cruz
Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# creating table for query 3
query_3_table_create = """
    CREATE TABLE IF NOT EXISTS music_library_3 (
        first_name text,
        last_name text,
        title text,
        PRIMARY KEY (title, first_name, last_name)
    );
"""

try:
    session.execute(query_3_table_create)
    print("Table 3 created successfully...")
except Exception as e:
    print(f"Exception encountered while creating table 3: {e}")

Table 3 created successfully...


In [15]:
# opening csv
file = 'event_datafile_full.csv'
query_3_table_insert = """
    INSERT INTO music_library_3 (first_name, last_name, title)
    VALUES (%s, %s, %s);
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    for line in csvreader:
        # database insertion
        session.execute(query_3_table_insert, (line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
select_query_3 = """
    SELECT first_name, last_name FROM music_library_3 
    WHERE title = 'All Hands Against His Own';
"""
try:
    rows = session.execute(select_query_3)
    for row in rows:
        print(f"{row.first_name}, {row.last_name}")
except Exception as e:
    print(f"Exception encountered while querying table 3: {e}")

Jacqueline, Lynch
Sara, Johnson
Tegan, Levine


### Drop the tables before closing out the sessions

In [17]:
# defining drop queries
query_1_table_drop = "DROP TABLE IF EXISTS music_library_1;"
query_2_table_drop = "DROP TABLE IF EXISTS music_library_2;"
query_3_table_drop = "DROP TABLE IF EXISTS music_library_3;"

In [18]:
try:
    session.execute(query_1_table_drop)
    session.execute(query_2_table_drop)
    session.execute(query_3_table_drop)
except Exception as e:
    print(f"Exception encountered while trying to drop tables: {e}")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()